# SQL Subqueries and Implementations

# Objectives

- Use SQL subqueries to nest queries
- Identify common SQL dialects and tools
- Query data from web databases

In [1]:
import pandas as pd
import sqlite3

# SQL Subqueries

Like you might nest one function within another in Python, you can nest queries in SQL. We can use a **subquery** within another query to succinctly implement queries that have multiple query steps.

In [2]:
conn = sqlite3.connect('flights.db')

## Subqueries in `FROM`

You can use a subquery in the `FROM` clause - this is useful, for example, if you want to apply multiple aggregation functions.

Let say we want to get the average of the number of routes departing from all airports. First we'd need to get the total number of routes departing from all airports, then take the average.

In [3]:
pd.read_sql('''
    SELECT 
        source AS depart_airport
        , COUNT() AS number_of_departures
    FROM
        routes
    GROUP BY
        source
''', conn)

,depart_airport,number_of_departures
0,AAE,9
1,AAL,20
2,AAN,2
3,AAQ,3
4,AAR,8
...,...,...
3404,ZUH,60
3405,ZUM,2
3406,ZVK,3
3407,ZYI,15


We can use this query as a subquery, and take the average of the new `number_of_departures` column.

In [4]:
pd.read_sql('''
    SELECT AVG(number_of_departures)
    FROM 
        (SELECT 
            source AS depart_airport
            , COUNT() AS number_of_departures
        FROM
            routes
        GROUP BY
            source)
''', conn)

,AVG(number_of_departures)
0,19.848343


## Subqueries in `WHERE`

You can use a subquery in the `WHERE` clause - this is useful, for example, if you want to filter a query based on results from another query.

Let's say that we want to get a table with all of the departures and destinations for the flight routes, but I only want to include flights departing from the five countries with the most airports.

To do this, we'd first need to identify the five countries that have the most airports. 

In [5]:
pd.read_sql('''
    SELECT 
        country,  
        COUNT() AS number_of_airports_in_country
    FROM
        airports
    GROUP BY
        country
    ORDER BY
        number_of_airports_in_country DESC
    LIMIT 5
''', conn)

,country,number_of_airports_in_country
0,United States,1697
1,Canada,435
2,Germany,321
3,Australia,263
4,Russia,249


I could enter these results into a new query of the routes table to get the data I want.

In [6]:
pd.read_sql('''
    SELECT 
        rt.source AS depart_airport, 
        rt.dest AS destination_airport, 
        ap.country AS depart_country
    FROM
        routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
    WHERE ap.country IN ("United States", "Canada", "Germany", "Australia", "Russia")
    ORDER BY depart_country
''', conn)

,depart_airport,destination_airport,depart_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


This approach works but has a few limitations:

- We have to manually enter the countries to filter them
- The list of countries won't update with our data, so we'd have to monitor and manually change them in the future
- We have to look at two separate queries to understand what our code is supposed to do
- We have to run two separate queries, which might take longer than one combined query

A better solution uses a subquery to get the list of 5 countries and feed it into our WHERE clause.

In [7]:
pd.read_sql('''
    SELECT 
        rt.source AS depart_airport, 
        rt.dest AS destination_airport, 
        ap.country AS depart_country
    FROM
        routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
    WHERE ap.country IN 
    -- Subquery to get the 5 countries with the most airports
    
        (SELECT country 
         FROM airports
         GROUP BY country
         ORDER BY COUNT() DESC
         LIMIT 5)
    
    ORDER BY depart_country
''', conn)

,depart_airport,destination_airport,depart_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


## Level Up: Common Table Expressions

Common Table Expressions (CTEs) are a more readable way to implement subqueries, using `WITH` and `AS`.

In [8]:
pd.read_sql('''

    WITH top_5_countries AS 
        (SELECT country 
         FROM airports
         GROUP BY country
         ORDER BY COUNT() DESC
         LIMIT 5) 
    
    SELECT 
        rt.source AS depart_airport, 
        rt.dest AS destination_airport, 
        ap.country AS depart_country
    FROM
        routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
    WHERE ap.country IN top_5_countries
    
    ORDER BY depart_country
''', conn)

,depart_airport,destination_airport,depart_country
0,DRW,SIN,Australia
1,PER,SIN,Australia
2,MEL,CTU,Australia
3,SYD,CKG,Australia
4,ADL,BNE,Australia
...,...,...,...
20330,SOW,FMN,United States
20331,SOW,PHX,United States
20332,SVC,PHX,United States
20333,VIS,LAX,United States


## Exercise

Create a table listing all airlines that serve the three airports with the most outbound routes.

<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    pd.read_sql('''

    WITH top_3_airports AS 
        (SELECT airports.id 
         FROM airports 
         LEFT JOIN routes
            ON routes.source_id = airports.id
         GROUP BY airports.id
         ORDER BY COUNT() DESC
         LIMIT 3) 
    
    SELECT DISTINCT
        rt.airline
    FROM
        routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id
    WHERE rt.source_id IN top_3_airports

    ''', conn)
    
</details>

In [39]:
pd.read_sql('''

WITH top_3_airports AS 
    (SELECT airports.id 
     FROM airports 
     LEFT JOIN routes
        ON routes.source_id = airports.id
     GROUP BY airports.id
     ORDER BY COUNT() DESC
     LIMIT 3) 

SELECT DISTINCT
    rt.airline
FROM
    routes AS rt
LEFT JOIN airports AS ap
    ON rt.source_id = ap.id
WHERE rt.source_id IN top_3_airports

''', conn)

,airline
0,3E
1,3M
2,3U
3,5J
4,8L
...,...
97,WN
98,WS
99,Y4
100,Y7


In [10]:
pd.read_sql('''
SELECT *
FROM airlines
''', conn).head(2)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N


In [11]:
pd.read_sql('''
SELECT *
FROM routes
''', conn).head(2)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2


In [14]:
pd.read_sql('''
SELECT *
FROM airports
''', conn).head(2)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


In [38]:
## Your work here
pd.read_sql('''

WITH top_3_ob_airport AS
    (SELECT id
    FROM
        routes AS rt
    LEFT JOIN airports AS ap
        ON rt.source_id = ap.id

    GROUP BY source
    ORDER BY COUNT(source) DESC
    LIMIT 3
    )

SELECT  DISTINCT airline as airline_name,
        ap.name as airport_name

FROM routes AS rt
LEFT JOIN airports AS ap
    ON rt.source_id = ap.id
WHERE source_id IN top_3_ob_airport

''', conn)

,airline_name,airport_name
0,3E,Chicago Ohare Intl
1,3M,Hartsfield Jackson Atlanta Intl
2,3U,Capital Intl
3,5J,Capital Intl
4,8L,Capital Intl
...,...,...
159,WS,Hartsfield Jackson Atlanta Intl
160,WS,Chicago Ohare Intl
161,Y4,Chicago Ohare Intl
162,Y7,Capital Intl


In [34]:
pd.read_sql('''
SELECT id, count(source), source, code
FROM
    routes AS rt
LEFT JOIN airports AS ap
    ON rt.source_id = ap.id
WHERE source = code
GROUP BY source
ORDER BY COUNT(source) DESC

''', conn)

,id,count(source),source,code
0,3682,915,ATL,ATL
1,3830,558,ORD,ORD
2,3364,535,PEK,PEK
3,507,527,LHR,LHR
4,1382,524,CDG,CDG
...,...,...,...,...
3265,1127,1,ABS,ABS
3266,5714,1,ABR,ABR
3267,3318,1,ABM,ABM
3268,3979,1,AAY,AAY


In [33]:
pd.read_sql('''

SELECT  airlines.name as airline_name,
        airports.name as airport_name,
        airports.id as airport_id

FROM airlines
JOIN airports USING(icao)
WHERE airport_id = '3364'
''', conn)

,airline_name,airport_name,airport_id


# SQL Versions

The is no one version of SQL - there are many versions out there! What you're learning about SQL with SQLite will apply to all of them. Just keep in mind when you apply for jobs that you may see any of these listed in any given job posting, and they are all just different versions of what you know.

## SQL Dialects

As with dialects of spoken languages, SQL dialects have many commonalities but some differences in syntax and functionality.  Here are a few of the major players:

- SQLite (we've already seen this!)
- PostgreSQL (free and open-source!)
- Oracle SQL
- MySQL (half open-souce, half Oracle)
- Microsoft SQL Server
- Transact-SQL (extends MS SQL)

## SQLite Pros & Cons

We use SQLite in this course, but it has some limitations.

### Pros

- Easy to set up
- Easy to share database files
- Uses little memory

### Cons

- Limited functionality for managing users and access permissions
- Not "thread safe": two edits at the same time can mess up your data

## Extra Resources: SQL Versions

[What Is a SQL Dialect, and Which one Should You Learn?](https://learnsql.com/blog/what-sql-dialect-to-learn/)

[SQLite vs MySQL vs PostgreSQL](https://www.digitalocean.com/community/tutorials/sqlite-vs-mysql-vs-postgresql-a-comparison-of-relational-database-management-systems)

[SQL Dialect Reference](https://en.wikibooks.org/wiki/SQL_Dialects_Reference)

# Web Databases: data.world

For the rest of this lesson, we'll be exploring databases in [data.world](https://data.world/), a web database that we can query using SQL in our browser. For reference, you can see the instructions for creating a new project here: [Getting Started Working with Data at data.world](https://help.data.world/hc/en-us/articles/360008853693-Getting-started-guide#working_with_data)

## Step 1: Create a data.world account

You will need to enter and verify your email address.

## Step 2: Create a project using this [Austin AirBnB](https://data.world/jonloyens/inside-airbnb-austin) dataset

Navigate to [this page](https://data.world/jonloyens/inside-airbnb-austin) and use the button at the top right of the page to create a new project using the data.

![](img/data_world_austin_airbnb_new_proj.png)

## Step 3: Create a SQL query

In your project, use the "+ Add" button to add a SQL query.

![](img/data_world_add_sql_query.png)

## Step 4: Run a simple SQL query

Try entering "SELECT * FROM listings" and clicking the "Run Query" button in the top right.

## Step 5: Practice using SQL to explore the data

Below are some exercises to practice your SQL skills and help explore the data. You will need to explore the table schemas to complete these exercises. 

You may find it helpful to click on the corresponding .csv files to inspect the data, or look at the columns in each table in the right sidebar.

### Exercise 1: Create a table showing the number of listings in each neighborhood
<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    SELECT neighbourhood, COUNT(*)
    FROM listings
    GROUP BY neighbourhood
    
</details>

### Exercise 2: Create a table showing the 20 listings with the most reviews
<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    SELECT *
    FROM listings
    ORDER BY number_of_reviews DESC
    LIMIT 20
    
</details>

### Exercise 3: Create a table showing all of the reviews for listings that are "Bed & Breakfast" property types.
<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    SELECT *
    FROM reviews
    INNER JOIN listings
        ON listings.id = reviews.listing_id
    WHERE listings.property_type = "Bed & Breakfast"
    
</details>

### Exercise 4: Run your own query using a subquery or CTE.

Note that the syntax and functionality for subqueries and CTEs in data.world are more limited than SQLite, so try creating simple ones.